In [ ]:
%run FedDeltaAlgo.ipynb

In [ ]:
from flask import Flask, request, render_template, render_template_string,jsonify
from werkzeug.exceptions import HTTPException
import os
import traceback

startupArticles()
app = Flask(__name__)


@app.route('/')
def index():
    return render_template("fedDelta.txt")

@app.route('/getCalendar')
def getCalendar():
    dates = []
    dateRegex = re.compile(r'(\d\d\d\d\d\d\d\d)')
    for datedFile in os.listdir('fedStates/'):
        if dateRegex.search(datedFile):
            dates.append(datedFile)
    dates.sort(reverse=True)
    res = jsonify(dates) 
    return res

@app.route('/ckeckUpdate')
def ckeckUpdate():
    res = dailyCheckArticle()
    res = jsonify(res)
    return res


@app.route('/readArticle')
def readArticle():
    try:
        sdate = request.args.get('sda')
        res =  extractArticle(sdate)
        res = jsonify(res) 
    except :
        res = jsonify(traceback.format_exc()) 
    return res

@app.route('/deltaArticles')
def deltaArticles():
    try:
        sdate0 = request.args.get('sda0')
        sdate1 = request.args.get('sda1')
        a0 =  extractArticle(sdate0)
        a1 =  extractArticle(sdate1)
    
        res = processArticle(splitArticle(a0), splitArticle(a1))
        res = jsonify(res) 
    except:
        res = jsonify(traceback.format_exc()) 
    return res

def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

@app.route('/quit')
def shutdown():
    shutdown_server()
    return 'Server shutting down...'

    
if __name__ == '__main__':
    #app.debug = True
    app.run() #debug=True 